In [1]:
PROJECT_ROOT = Path("C:/Users/bo_co/ai-sports-competitive-intel/nba")


NameError: name 'Path' is not defined

In [ ]:
from pathlib import Path
print("Working directory:", Path.cwd())


In [ ]:
import pandas as pd
from pathlib import Path

DATA_PATH = Path(
    "C:/Users/bo_co/ai-sports-competitive-intel/nba/data/raw/player_stats_2024_25.csv.gz"
)

assert DATA_PATH.exists(), f"Missing file: {DATA_PATH}"
df = pd.read_csv(DATA_PATH)

print("Loaded:", df.shape)
df.head()



In [ ]:
from pathlib import Path
Path.cwd()


In [ ]:
from pathlib import Path

hits = list(Path.home().rglob("player_stats_2024_25.csv*"))
hits


In [ ]:
df["GAME_DATE"] = pd.to_datetime(df["gameDateTimeEst"], errors="coerce")
df["PLAYER_NAME"] = df["firstName"].str.strip() + " " + df["lastName"].str.strip()

df = df.rename(columns={
    "numMinutes": "MIN",
    "points": "PTS",
    "reboundsTotal": "REB",
    "assists": "AST"
})

df[["PLAYER_NAME","GAME_DATE","MIN","PTS","REB","AST"]].head(3)


In [ ]:
def fetch_player_season_logs_local(player_name: str) -> pd.DataFrame:
    sub = df[df["PLAYER_NAME"].str.lower() == player_name.lower()].copy()
    if sub.empty:
        first_token = player_name.lower().split()[0]
        suggestions = (
            df[df["PLAYER_NAME"].str.lower().str.contains(first_token, na=False)]
            ["PLAYER_NAME"].drop_duplicates().head(15).tolist()
        )
        raise ValueError(f"No rows found for player: {player_name}. Suggestions: {suggestions}")

    return sub.sort_values("GAME_DATE").reset_index(drop=True)


In [ ]:
def teammate_impact_local_v2(player_name: str, teammate_name: str, stats_cols=None) -> pd.DataFrame:
    if stats_cols is None:
        stats_cols = ["MIN", "PTS", "REB", "AST"]

    p_df = fetch_player_season_logs_local(player_name)
    t_df = fetch_player_season_logs_local(teammate_name)

    teammate_dates = set(t_df["GAME_DATE"])
    with_tm = p_df[p_df["GAME_DATE"].isin(teammate_dates)]
    without_tm = p_df[~p_df["GAME_DATE"].isin(teammate_dates)]

    n_with = len(with_tm)
    n_without = len(without_tm)

    out = pd.DataFrame({
        "with_avg": with_tm[stats_cols].mean(),
        "without_avg": without_tm[stats_cols].mean(),
        "delta_without_minus_with": without_tm[stats_cols].mean() - with_tm[stats_cols].mean(),
        "with_std": with_tm[stats_cols].std(),
        "without_std": without_tm[stats_cols].std(),
    })

    out["n_games_with"] = n_with
    out["n_games_without"] = n_without
    out["low_sample_flag"] = n_without < 10

    return out


In [ ]:
teammate_impact_local_v2("Brandin Podziemski", "Stephen Curry")


In [ ]:
def get_teammates(star_name: str) -> list[str]:
    star_df = fetch_player_season_logs_local(star_name)

    # Steph’s team IDs (handles trades safely)
    star_team_ids = set(star_df["teamId"])

    # Games Steph played
    game_ids = set(star_df["gameId"])

    teammates = (
        df[
            (df["gameId"].isin(game_ids)) &
            (df["teamId"].isin(star_team_ids))
        ]["PLAYER_NAME"]
        .drop_duplicates()
        .tolist()
    )

    teammates = [t for t in teammates if t.lower() != star_name.lower()]
    return sorted(teammates)


In [ ]:
get_teammates("Stephen Curry")


In [ ]:
[c for c in df.columns if "team" in c.lower()]


In [ ]:
def get_teammates(star_name: str) -> list[str]:
    star_df = fetch_player_season_logs_local(star_name)

    # Team identifiers for the star (handles trades)
    star_team_names = set(star_df["playerteamName"].dropna().unique())
    star_team_cities = set(star_df["playerteamCity"].dropna().unique())

    game_ids = set(star_df["gameId"])

    teammates = (
        df[
            (df["gameId"].isin(game_ids)) &
            (df["playerteamName"].isin(star_team_names)) &
            (df["playerteamCity"].isin(star_team_cities))
        ]["PLAYER_NAME"]
        .drop_duplicates()
        .tolist()
    )

    teammates = [t for t in teammates if t.lower() != star_name.lower()]
    return sorted(teammates)


In [ ]:
get_teammates("Stephen Curry")[:25]


In [ ]:
fetch_player_season_logs_local("Stephen Curry")[["playerteamCity","playerteamName"]].drop_duplicates()


In [ ]:
def teammate_impact_local_v3(player_name: str, star_name: str, stats_cols=None) -> pd.DataFrame:
    if stats_cols is None:
        stats_cols = ["MIN", "PTS", "REB", "AST"]

    p_df = fetch_player_season_logs_local(player_name)
    s_df = fetch_player_season_logs_local(star_name)

    star_game_ids = set(s_df["gameId"])

    with_star = p_df[p_df["gameId"].isin(star_game_ids)]
    without_star = p_df[~p_df["gameId"].isin(star_game_ids)]

    n_with = len(with_star)
    n_without = len(without_star)

    out = pd.DataFrame({
        "with_avg": with_star[stats_cols].mean(),
        "without_avg": without_star[stats_cols].mean(),
        "delta_without_minus_with": without_star[stats_cols].mean() - with_star[stats_cols].mean(),
        "with_std": with_star[stats_cols].std(),
        "without_std": without_star[stats_cols].std(),
    })
    out["n_games_with"] = n_with
    out["n_games_without"] = n_without
    out["low_sample_flag"] = n_without < 10
    return out


In [ ]:
get_teammates("Stephen Curry")[:15]


In [ ]:
def teammate_impact_local_v3(player_name: str, star_name: str, stats_cols=None) -> pd.DataFrame:
    if stats_cols is None:
        stats_cols = ["MIN", "PTS", "REB", "AST"]

    p_df = fetch_player_season_logs_local(player_name)
    s_df = fetch_player_season_logs_local(star_name)

    star_game_ids = set(s_df["gameId"])

    with_star = p_df[p_df["gameId"].isin(star_game_ids)]
    without_star = p_df[~p_df["gameId"].isin(star_game_ids)]

    n_with = len(with_star)
    n_without = len(without_star)

    out = pd.DataFrame({
        "with_avg": with_star[stats_cols].mean(),
        "without_avg": without_star[stats_cols].mean(),
        "delta_without_minus_with": without_star[stats_cols].mean() - with_star[stats_cols].mean(),
        "with_std": with_star[stats_cols].std(),
        "without_std": without_star[stats_cols].std(),
    })
    out["n_games_with"] = n_with
    out["n_games_without"] = n_without
    out["low_sample_flag"] = n_without < 10
    return out


In [ ]:
teammate_impact_local_v3("Brandin Podziemski", "Stephen Curry")


In [ ]:
def top_fantasy_adds_when_out_v2(
    star_name: str,
    stat_focus: str = "PTS",
    min_games_without: int = 5
) -> pd.DataFrame:

    teammates = get_teammates(star_name)
    rows = []

    for tm in teammates:
        try:
            res = teammate_impact_local_v3(tm, star_name)
            n_without = int(res.loc[stat_focus, "n_games_without"])
            if n_without < min_games_without:
                continue

            rows.append({
                "PLAYER": tm,
                "WITH_AVG": float(res.loc[stat_focus, "with_avg"]),
                "WITHOUT_AVG": float(res.loc[stat_focus, "without_avg"]),
                "DELTA": float(res.loc[stat_focus, "delta_without_minus_with"]),
                "N_WITHOUT": n_without,
                "LOW_SAMPLE": bool(res.loc[stat_focus, "low_sample_flag"]),
            })
        except Exception:
            continue

    out = pd.DataFrame(rows)
    if out.empty:
        return out

    return out.sort_values("DELTA", ascending=False).reset_index(drop=True)


In [ ]:
top_fantasy_adds_when_out_v2("Stephen Curry", stat_focus="PTS", min_games_without=5).head(15)


In [ ]:
def top_fantasy_adds_composite(
    star_name: str,
    min_games_without: int = 5,
    weights=None
) -> pd.DataFrame:

    if weights is None:
        weights = {"PTS": 1.0, "AST": 0.75, "REB": 0.5, "MIN": 0.10}

    teammates = get_teammates(star_name)
    rows = []

    for tm in teammates:
        try:
            res = teammate_impact_local_v3(tm, star_name)

            # enforce minimum sample based on MIN bucket (same counts across stats, but keep it explicit)
            n_without = int(res.loc["MIN", "n_games_without"])
            if n_without < min_games_without:
                continue

            deltas = {stat: float(res.loc[stat, "delta_without_minus_with"]) for stat in weights.keys()}
            score = sum(weights[k] * deltas[k] for k in weights.keys())

            rows.append({
                "PLAYER": tm,
                "SCORE": score,
                "DELTA_PTS": deltas["PTS"],
                "DELTA_AST": deltas["AST"],
                "DELTA_REB": deltas["REB"],
                "DELTA_MIN": deltas["MIN"],
                "N_WITHOUT": n_without,
                "LOW_SAMPLE": bool(res.loc["MIN", "low_sample_flag"]),
            })
        except Exception:
            continue

    out = pd.DataFrame(rows)
    if out.empty:
        return out

    return out.sort_values("SCORE", ascending=False).reset_index(drop=True)


In [ ]:
top_fantasy_adds_composite("Stephen Curry", min_games_without=5).head(15)


In [ ]:
from pathlib import Path

PROJECT_ROOT = Path("C:/Users/bo_co/ai-sports-competitive-intel/nba")
REPORTS_DIR = PROJECT_ROOT / "data" / "reports"
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

def add_confidence_label(out: pd.DataFrame) -> pd.DataFrame:
    out = out.copy()
    def label(n):
        if n >= 30: return "High"
        if n >= 15: return "Medium"
        return "Low"
    out["CONFIDENCE"] = out["N_WITHOUT"].apply(label)
    return out

report = add_confidence_label(
    top_fantasy_adds_composite("Stephen Curry", min_games_without=5)
)

csv_out = REPORTS_DIR / "warriors_adds_when_curry_out_2024_25.csv"
report.to_csv(csv_out, index=False)

csv_out.resolve()



In [ ]:
html_out = REPORTS_DIR / "warriors_adds_when_curry_out_2024_25.html"

top10 = report.head(10).copy()

for c in ["SCORE","DELTA_PTS","DELTA_AST","DELTA_REB","DELTA_MIN"]:
    top10[c] = top10[c].map(lambda x: f"{x:.2f}")

html = f"""
<html>
<head>
  <meta charset="utf-8">
  <title>Warriors: Top Fantasy Adds When Steph Curry Is Out (2024–25)</title>
  <style>
    body {{ font-family: Arial, sans-serif; margin: 24px; }}
    h1 {{ margin-bottom: 6px; }}
    .note {{ color: #444; margin-bottom: 18px; }}
    table {{ border-collapse: collapse; width: 100%; }}
    th, td {{ border: 1px solid #ddd; padding: 8px; text-align: left; }}
    th {{ background: #f4f4f4; }}
  </style>
</head>
<body>
  <h1>Warriors: Top Fantasy Adds When Steph Curry Is Out (2024–25)</h1>
  <div class="note">
    Score = 1.0×ΔPTS + 0.75×ΔAST + 0.5×ΔREB + 0.10×ΔMIN.<br>
    Confidence based on number of games without Curry.
  </div>
  {top10.to_html(index=False, escape=True)}
</body>
</html>
"""

html_out.write_text(html, encoding="utf-8")
html_out.resolve()


In [2]:
from pathlib import Path
print("CWD:", Path.cwd())


CWD: C:\Users\bo_co\ai-sports-competitive-intel\nba\notebooks
